In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [12]:
data_set = '/content/drive/MyDrive/dogs'

data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = data_gen.flow_from_directory(
    data_set,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_gen.flow_from_directory(
    data_set,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='validation'
)


Found 64 images belonging to 4 classes.
Found 16 images belonging to 4 classes.


In [45]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

conv_base.trainable = False

model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size if train_generator.samples // train_generator.batch_size > 0 else 1,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size if validation_generator.samples // validation_generator.batch_size > 0 else 1
)

Epoch 1/30
3/3 [==============================] - 6s 1s/step - loss: 4.7852 - accuracy: 0.2045 - val_loss: 2.6138 - val_accuracy: 0.3125
Epoch 2/30
3/3 [==============================] - 3s 1s/step - loss: 3.3436 - accuracy: 0.2833 - val_loss: 3.1071 - val_accuracy: 0.2500
Epoch 3/30
3/3 [==============================] - 3s 1s/step - loss: 2.8505 - accuracy: 0.2500 - val_loss: 1.3122 - val_accuracy: 0.5000
Epoch 4/30
3/3 [==============================] - 4s 2s/step - loss: 1.9263 - accuracy: 0.3864 - val_loss: 1.6815 - val_accuracy: 0.3750
Epoch 5/30
3/3 [==============================] - 3s 1s/step - loss: 1.6257 - accuracy: 0.4545 - val_loss: 1.5912 - val_accuracy: 0.2500
Epoch 6/30
3/3 [==============================] - 3s 1s/step - loss: 1.8664 - accuracy: 0.4318 - val_loss: 1.0403 - val_accuracy: 0.6875
Epoch 7/30
3/3 [==============================] - 3s 1s/step - loss: 1.2232 - accuracy: 0.5227 - val_loss: 2.1219 - val_accuracy: 0.2500
Epoch 8/30
3/3 [=========================

In [51]:
import numpy as np
from tensorflow.keras.preprocessing import image

class_labels = ['French bulldog', 'German shepherd', 'Golden retriever', 'Poodle']

def predict_image(img_path, model, threshold=0.6):
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    prediction = model.predict(img_tensor)
    pred_class = np.argmax(prediction, axis=1)[0]
    pred_label = class_labels[pred_class]

    if np.max(prediction) < threshold:
        pred_label = 'other'
    return pred_label

img_path = '/content/drive/MyDrive/7 10 2017/Mixed-Breeds-Of-Poodles-Best-scaled-e1586976423144.jpg'
prediction = predict_image(img_path, model)
print(f'Prediction: {prediction}')


1/1 [==============================] - 0s 18ms/step
Prediction: Poodle
